In [3]:
#importing packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

import os
os.listdir("test_images/")

from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Read in and grayscale the image 
#image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
def grayscale_process(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    return gray

# Define a kernel size and apply Gaussian smoothing
def gaussian_blur_process(gray):
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    return blur_gray

# Define our parameters for Canny and apply
def canny_process(blur_gray):
    low_threshold = 50
    high_threshold = 200
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

# Next we'll create a masked edges image using cv2.fillPoly()
def masked_edges_image(image, edges):
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   
    
    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(150,imshape[0]),(400, 322), (550, 322), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges

# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
def hough_lines_process(masked_edges, rho, theta, threshold, min_line_length, max_line_gap):
    hough = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    return hough

rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 40  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 15#minimum number of pixels making up a line
max_line_gap = 150#maximum gap in pixels between connectable line segments  

def weight_img(lines_img, image):
    return cv2.addWeighted(image, 1, lines_img, 1, 0)

def draw_lines(image, hough):
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines_img = np.copy(image)*0 # Black image
    lines = hough
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(lines_img,(x1,y1),(x2,y2),(255,0,0),10)
    lines_edges = weight_img(lines_img, image)
    return lines_edges 

def process_image(image):
        # Grayscale
        gray = grayscale_process(image)
        # Gaussian Smoothing
        blur_gray = gaussian_blur_process(gray)
        # Canny Edge
        edges = canny_process(blur_gray)
        # Region of Interest
        masked_edges = masked_edges_image(image, edges)
        # Hough Transform
        hough = hough_lines_process(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
        # Draw lines 
        lines_edges = draw_lines(image, hough) 
        return lines_edges

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:14<00:00, 15.60it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.21 s, sys: 235 ms, total: 3.44 s
Wall time: 16 s
